In [1]:
%load_ext autotime
import pandas as pd
import os
import geopandas as gpd
from glob import glob
from shapely.validation import make_valid
from tqdm.auto import tqdm
import time
pd.set_option("display.max_colwidth", 100)

In [2]:
files = pd.DataFrame({"filepath": glob("Asset Directory/**", recursive=True)})
files = files[files["filepath"].str.endswith((".shp", ".zip", ".gpkg", ".geojson"))]
files["filename"] = files["filepath"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
files["filesize_GB"] = files["filepath"].apply(lambda x: os.path.getsize(x) / 1e9)
files.sort_values("filesize_GB", ascending=False).reset_index(drop=True)

,filepath,filename,filesize_GB
0,Asset Directory/LUCAS NZ Land Use/lucas-nz-land-use-map-1990-2008-2012-2016-v011.shp,lucas-nz-land-use-map-1990-2008-2012-2016-v011,1.435426
1,Asset Directory/Koordinate/smap-soil-drainage-dec-2023/smap-soil-drainage-dec-2023.shp,smap-soil-drainage-dec-2023,0.699233
2,Asset Directory/lds-new-zealand-17layers-SHP/nz-building-outlines/nz-building-outlines.shp,nz-building-outlines,0.636510
3,Asset Directory/lris-fsl-soil-drainage-class-SHP/fsl-soil-drainage-class.shp,fsl-soil-drainage-class,0.164699
4,Asset Directory/Koordinate/nzlri-land-use-capability-2021/nzlri-land-use-capability-2021.shp,nzlri-land-use-capability-2021,0.164578
5,Asset Directory/DOC/DOC_Public_Conservation_Land.shp,DOC_Public_Conservation_Land,0.139802
6,Asset Directory/Koordinate/protected-areas/protected-areas.shp,protected-areas,0.134675
7,Asset Directory/NationalRoadCentrelineDataset/roads.shp,roads,0.130881
8,Asset Directory/lds-new-zealand-17layers-SHP/nz-road-centrelines-topo-150k.shp,nz-road-centrelines-topo-150k,0.116422
9,Asset Directory/lds-new-zealand-17layers-SHP/nz-track-centrelines-topo-150k/nz-track-centrelines...,nz-track-centrelines-topo-150k,0.074558


In [3]:
polys = []
for f in tqdm(glob("Projections/*_polygon.shp")):
  df = gpd.read_file(f)
  df.geometry = df.geometry.apply(make_valid)
  polys.append(df.union_all())

  0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
os.unlink("affected_assets.gpkg")
for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = pd.concat(gpd.clip(asset, poly) for poly in tqdm(polys))
  print(f"Time to clip: {time.time() - start}")
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("affected_assets.gpkg", layer=row.filename, driver='GPKG')

  0%|          | 0/42 [00:00<?, ?it/s]

Now processing lucas-nz-land-use-map-1990-2008-2012-2016-v011
Time to read file: 19.914244413375854


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 56.55389451980591
Now processing fsl-soil-drainage-class
Time to read file: 1.984891653060913


  0%|          | 0/6 [00:00<?, ?it/s]

...

Time to read file: 16.471911430358887


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 14.167056322097778
Now processing nzlri-land-use-capability-2021
Time to read file: 4.651247024536133


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 10.393149852752686


In [5]:
gpd.list_layers("affected_assets.gpkg")

,name,geometry_type
0,lucas-nz-land-use-map-1990-2008-2012-2016-v011,MultiPolygon
1,fsl-soil-drainage-class,MultiPolygon
2,Transpower Transmission Lines,Unknown
3,generation_NZ,Unknown
4,wwtp,Unknown
5,wtp,Unknown
6,DOC_Marine_Reserves,MultiPolygon
7,Ecological_Districts,MultiPolygon
8,Internationally_Recognised_Areas,MultiPolygon
9,DOC_Sanctuaries_to_Protect_Marine_Mammals,MultiPolygon


In [6]:
files = files[~files.filename.isin(gpd.list_layers("affected_assets.gpkg").name)]
files

,filepath,filename,filesize_GB


In [7]:
for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath).to_crs(2193)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = pd.concat(gpd.clip(asset, poly) for poly in tqdm(polys))
  print(f"Time to clip: {time.time() - start}")
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("affected_assets.gpkg", layer=row.filename, driver='GPKG')

0it [00:00, ?it/s]